In [1]:
import pandas as pd
import numpy as np
import cplex
from cplex.exceptions import CplexError

In [2]:
# Load data from the Excel file
file_path = 'OriginalDataset.xlsx'
data = pd.read_excel(file_path, sheet_name=None)

In [3]:
sets_df = data.get('Sets', pd.DataFrame())
warehouse = sets_df[sets_df['Type'] == 'Warehouse']
vehicle = sets_df[sets_df['Type'] == 'Vehicle']
customer = sets_df[sets_df['Type'] == 'Customer']
pickup = sets_df[sets_df['Type'] == 'Pickup']
delivery = sets_df[sets_df['Type'] == 'Delivery']

In [4]:
delivery

,Type,ID
25,Delivery,C1
26,Delivery,C2
27,Delivery,C3
28,Delivery,C4
29,Delivery,C5
30,Delivery,C6
31,Delivery,C7
32,Delivery,C8
33,Delivery,C9
34,Delivery,C10


In [5]:
transportcost = data.get('TransportationCost', pd.DataFrame())
carbon_emissions = data.get('CarbonEmissions', pd.DataFrame())
pickupdemands = data.get('PickupDemand', pd.DataFrame())
deliverydemands = data.get('DeliveryDemand', pd.DataFrame())
vehicle_capacity = data.get('VehicleCapacity', pd.DataFrame())

n_customers = 16
Ccustomer = customer.head(n_customers)
Cpickup = pickup[pickup['ID'].isin(Ccustomer['ID'])]
Cdelivery = delivery[delivery['ID'].isin(Ccustomer['ID'])]

Ce = carbon_emissions[carbon_emissions['Unnamed: 0'].isin(Ccustomer['ID'])]

# Sets and indices
W = set(warehouse['ID'])
K = set(vehicle['ID'])
C = set(Ccustomer['ID'])
Cd = set(Cdelivery['ID'])
Cp = set(Cpickup['ID'])

delivery_list = Cdelivery['ID'].tolist()  # List of delivery locations
pickup_list = Cpickup['ID'].tolist()     # List of pickup locations

vehicle_routes = {}

for k in K:
    vehicle_routes[k] = {
        'deliveries': delivery_list.copy(),  # Copy of delivery locations for vehicle k
        'pickups': pickup_list.copy()        # Copy of pickup locations for vehicle k
    }

In [6]:
vehicle_routes

{'V2': {'deliveries': ['C1',
   'C2',
   'C3',
   'C4',
   'C5',
   'C6',
   'C7',
   'C8',
   'C9',
   'C10',
   'C11',
   'C12',
   'C13',
   'C14',
   'C15',
   'C16'],
  'pickups': ['C3', 'C6', 'C11', 'C13']},
 'V1': {'deliveries': ['C1',
   'C2',
   'C3',
   'C4',
   'C5',
   'C6',
   'C7',
   'C8',
   'C9',
   'C10',
   'C11',
   'C12',
   'C13',
   'C14',
   'C15',
   'C16'],
  'pickups': ['C3', 'C6', 'C11', 'C13']}}

In [7]:
# Parameters
locations = list(C.union(W))
cij = {}
eij = {}
for k in K:
    for i in locations:
        for j in locations:
            cij[(i, j, k)] = float(transportcost.loc[transportcost.iloc[:, 0] == i, str(j)].values[0])
            eij[(i, j, k)] = float(carbon_emissions.loc[carbon_emissions.iloc[:, 0] == i, str(j)].values[0])
            

In [8]:
eij

{('C2', 'C2', 'V2'): 0.364495,
 ('C2', 'C11', 'V2'): 0.3681611092,
 ('C2', 'C13', 'V2'): 0.3678791008,
 ('C2', 'C5', 'V2'): 0.3691011372,
 ('C2', 'C6', 'V2'): 0.3682081106,
 ('C2', 'C15', 'V2'): 0.3683021134,
 ('C2', 'C4', 'V2'): 0.3680671064,
 ('C2', 'C16', 'V2'): 0.368020105,
 ('C2', 'W1', 'V2'): 0.3691011372,
 ('C2', 'C1', 'V2'): 0.3675970924,
 ('C2', 'C9', 'V2'): 0.3690071344,
 ('C2', 'C8', 'V2'): 0.368255112,
 ('C2', 'C12', 'V2'): 0.3688661302,
 ('C2', 'C3', 'V2'): 0.3665160602,
 ('C2', 'C7', 'V2'): 0.3667510672,
 ('C2', 'C10', 'V2'): 0.3661870504,
 ('C2', 'C14', 'V2'): 0.3681141078,
 ('C11', 'C2', 'V2'): 0.3681611092,
 ('C11', 'C11', 'V2'): 0.364495,
 ('C11', 'C13', 'V2'): 0.3654820294,
 ('C11', 'C5', 'V2'): 0.3686311232,
 ('C11', 'C6', 'V2'): 0.3690071344,
 ('C11', 'C15', 'V2'): 0.3669390728,
 ('C11', 'C4', 'V2'): 0.3683961162,
 ('C11', 'C16', 'V2'): 0.3682081106,
 ('C11', 'W1', 'V2'): 0.3683021134,
 ('C11', 'C1', 'V2'): 0.368255112,
 ('C11', 'C9', 'V2'): 0.3651060182,
 ('C11', 

In [9]:
di = deliverydemands.set_index('CustomerID').to_dict()['Demand']
pi = pickupdemands.set_index('CustomerID').to_dict()['Demand']
Qk = vehicle_capacity.set_index('VehicleID').to_dict()['Capacity']

In [10]:
pi

{'C3': 1.531184,
 'C6': 6.93072,
 'C11': 0.343479,
 'C13': 3.129383,
 'C19': 0.503502}

In [11]:
alpha = 0.0733
Emax = 1000

In [12]:
# Initialize the model
model = cplex.Cplex()
model.set_problem_type(cplex.Cplex.problem_type.MILP)

# Decision variables
x = {}
u = {}

for k in K:
    for i in locations:
        for j in locations:
            var_name = f"x_{i}_{j}_{k}"
            x[(i, j, k)] = var_name
            model.variables.add(names=[var_name], types=[model.variables.type.binary])
        u[(i, k)] = f"u_{i}_{k}"
        model.variables.add(names=[u[(i, k)]], types=[model.variables.type.continuous])
        
# Adjust the objective function to minimize both cost and emissions
objective_vars = []
objective_coeffs = []

for k in K:
    for i in C:
        for j in C:
            if i != j:
                objective_vars.append(x[(i, j, k)])
                objective_coeffs.append(cij[(i, j, k)] + eij[(i, j, k)])  # Combine cost and emissions

# Minimize the objective
model.objective.set_linear(zip(objective_vars, objective_coeffs))
model.objective.set_sense(model.objective.sense.minimize)

In [13]:
locations

['C2',
 'C11',
 'C13',
 'C5',
 'C6',
 'C15',
 'C4',
 'C16',
 'W1',
 'C1',
 'C9',
 'C8',
 'C12',
 'C3',
 'C7',
 'C10',
 'C14']

In [14]:
# Each customer is visited exactly once by one vehicle
for i in C:  # Visiting each customer
    lhs_vars = [x[(j, i, k)] for j in locations for k in K if j != i]  # Variables indicating visits to customer i
    lhs_coeffs = [1] * len(lhs_vars)  # Coefficients for the linear expression (all 1s)
    if i in vehicle_routes[k]['deliveries'] and i in vehicle_routes[k]['pickups']:
        rhs_value = 2  # Allow two visits for customers with both delivery and pickup
    else:
        rhs_value = 1  # Otherwise, just one visit
    print(f"Customer {i} visit constraint: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={rhs_value}")
    name = f"Customer_visited_{i}"
    model.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
        senses=["E"],  # Equality constraint
        rhs=[rhs_value], names=[name]
    )

# Vehicles must start and end at a warehouse
for k in K:
    start_constraint_expr = cplex.SparsePair(ind=[], val=[])
    for w in W:
        for j in C:
            start_constraint_expr.ind.append(x[(w, j, k)])
            start_constraint_expr.val.append(1)
    print(f"Vehicle {k} start constraint: vars={start_constraint_expr.ind}, coeffs={start_constraint_expr.val}, rhs=1")
    name = "Start at W"
    model.linear_constraints.add(
        lin_expr=[start_constraint_expr],
        senses=["E"], rhs=[1],names=[name]
    )
        
for k in K:
    end_constraint_expr = cplex.SparsePair(ind=[], val=[])
    for w in W:
        for i in C:            
            end_constraint_expr.ind.append(x[(i, w, k)])
            end_constraint_expr.val.append(1)
    print(f"Vehicle {k} end constraint: vars={end_constraint_expr.ind}, coeffs={end_constraint_expr.val}, rhs=1")
    name = "End at W"
    model.linear_constraints.add(
        lin_expr=[end_constraint_expr],
        senses=["E"], rhs=[1], names=[name]
    )

Customer C2 visit constraint: vars=['x_C11_C2_V2', 'x_C11_C2_V1', 'x_C13_C2_V2', 'x_C13_C2_V1', 'x_C5_C2_V2', 'x_C5_C2_V1', 'x_C6_C2_V2', 'x_C6_C2_V1', 'x_C15_C2_V2', 'x_C15_C2_V1', 'x_C4_C2_V2', 'x_C4_C2_V1', 'x_C16_C2_V2', 'x_C16_C2_V1', 'x_W1_C2_V2', 'x_W1_C2_V1', 'x_C1_C2_V2', 'x_C1_C2_V1', 'x_C9_C2_V2', 'x_C9_C2_V1', 'x_C8_C2_V2', 'x_C8_C2_V1', 'x_C12_C2_V2', 'x_C12_C2_V1', 'x_C3_C2_V2', 'x_C3_C2_V1', 'x_C7_C2_V2', 'x_C7_C2_V1', 'x_C10_C2_V2', 'x_C10_C2_V1', 'x_C14_C2_V2', 'x_C14_C2_V1'], coeffs=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], rhs=1
Customer C10 visit constraint: vars=['x_C2_C10_V2', 'x_C2_C10_V1', 'x_C11_C10_V2', 'x_C11_C10_V1', 'x_C13_C10_V2', 'x_C13_C10_V1', 'x_C5_C10_V2', 'x_C5_C10_V1', 'x_C6_C10_V2', 'x_C6_C10_V1', 'x_C15_C10_V2', 'x_C15_C10_V1', 'x_C4_C10_V2', 'x_C4_C10_V1', 'x_C16_C10_V2', 'x_C16_C10_V1', 'x_W1_C10_V2', 'x_W1_C10_V1', 'x_C1_C10_V2', 'x_C1_C10_V1', 'x_C9_C10_V2', 'x_C9_C10_V1', 'x_C8_C10_V2', 

In [15]:
# Flow conservation
for k in K:
    for i in C:
        incoming = [x[(j, i, k)] for j in locations if j != i]
        outgoing = [x[(i, j, k)] for j in locations if j != i]
        print(f"Flow conservation for vehicle {k} at location {i}: incoming={incoming}, outgoing={outgoing}")
        name="Flow Conservation"   
        model.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=incoming + outgoing, val=[1] * len(incoming) + [-1] * len(outgoing))],
                senses=["E"],
                rhs=[0],names=[name]
            )
        
# MTZ Subtour elimination
for k in K:
    for i in C:
        for j in C:
            lhs_vars=[]
            lhs_coeffs=[]
            if i != j:
                if j not in pi:
                    pi[j] = 0
                if j not in di:
                    di[j] = 0
                lhs_vars.append("u_" +str(i)+"_"+str(k))
                lhs_coeffs.append(1)
                lhs_vars.append("u_" +str(j)+"_"+str(k))
                lhs_coeffs.append(-1)
                lhs_vars.append("x_" +str(i)+"_"+str(j)+"_"+str(k))
                lhs_coeffs.append(Qk[k])
                rhs = Qk[k] - di[j] + pi[j]
                print(f"MTZ constraint for vehicle {k} from {i} to {j}: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={rhs}")
                name = "Subtour1"
                model.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
                    senses=["L"],
                    rhs=[rhs], names=[name]
                )
            
# Capacity constraint
for k in K:
    for i in C:
        lhs_vars=[]
        lhs_coeffs=[]
        if i in pi:
            pickup = pi[i]
        else:
            pickup = 0
        if i in di:
            delivery = di[i]
        else:
            delivery = 0

        lhs_vars.append("u_" +str(i)+"_"+str(k))
        lhs_coeffs.append(1)
        print(f"Capacity constraint for vehicle {k} at customer {i}: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={Qk[k]}")
        name = "capacity"
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
            senses=["L"],
            rhs=[Qk[k]], names=[name]
        )
    
model.parameters.mip.tolerances.mipgap.set(0.01)  # For example, set MIP gap tolerance

Flow conservation for vehicle V2 at location C2: incoming=['x_C11_C2_V2', 'x_C13_C2_V2', 'x_C5_C2_V2', 'x_C6_C2_V2', 'x_C15_C2_V2', 'x_C4_C2_V2', 'x_C16_C2_V2', 'x_W1_C2_V2', 'x_C1_C2_V2', 'x_C9_C2_V2', 'x_C8_C2_V2', 'x_C12_C2_V2', 'x_C3_C2_V2', 'x_C7_C2_V2', 'x_C10_C2_V2', 'x_C14_C2_V2'], outgoing=['x_C2_C11_V2', 'x_C2_C13_V2', 'x_C2_C5_V2', 'x_C2_C6_V2', 'x_C2_C15_V2', 'x_C2_C4_V2', 'x_C2_C16_V2', 'x_C2_W1_V2', 'x_C2_C1_V2', 'x_C2_C9_V2', 'x_C2_C8_V2', 'x_C2_C12_V2', 'x_C2_C3_V2', 'x_C2_C7_V2', 'x_C2_C10_V2', 'x_C2_C14_V2']
Flow conservation for vehicle V2 at location C10: incoming=['x_C2_C10_V2', 'x_C11_C10_V2', 'x_C13_C10_V2', 'x_C5_C10_V2', 'x_C6_C10_V2', 'x_C15_C10_V2', 'x_C4_C10_V2', 'x_C16_C10_V2', 'x_W1_C10_V2', 'x_C1_C10_V2', 'x_C9_C10_V2', 'x_C8_C10_V2', 'x_C12_C10_V2', 'x_C3_C10_V2', 'x_C7_C10_V2', 'x_C14_C10_V2'], outgoing=['x_C10_C2_V2', 'x_C10_C11_V2', 'x_C10_C13_V2', 'x_C10_C5_V2', 'x_C10_C6_V2', 'x_C10_C15_V2', 'x_C10_C4_V2', 'x_C10_C16_V2', 'x_C10_W1_V2', 'x_C10_C1_V2

In [16]:
#Emissions constraint 
lhs_vars = []
lhs_coeffs = []

# Add contributions of eij * xijk
for k in K:
    for i in C:  # Use locations to include both warehouses and customers
        for j in C:
            if i != j:
                lhs_vars.append(x[(i, j, k)])
                lhs_coeffs.append(eij[(i, j, k)])
                print(f"Adding x[{i}, {j}, {k}] with coefficient {eij[(i, j, k)]}")

# Add contributions of alpha * dij * uik
for k in K:
    for i in C:
        if i in di:
            delivery = di[i]
        else:
            delivery = 0
        lhs_vars.append(u[(i, k)])
        lhs_coeffs.append(alpha * delivery)
        print(f"Adding u[{i}, {k}] with coefficient {alpha * delivery}")

print(f"Total emission constraint: vars={lhs_vars}, coeffs={lhs_coeffs}, rhs={Emax}")
name = "Emissions"
model.linear_constraints.add(
    lin_expr=[cplex.SparsePair(ind=lhs_vars, val=lhs_coeffs)],
    senses=["L"],
    rhs=[Emax], names=[name]
)

Adding x[C2, C10, V2] with coefficient 0.3661870504
Adding x[C2, C4, V2] with coefficient 0.3680671064
Adding x[C2, C15, V2] with coefficient 0.3683021134
Adding x[C2, C16, V2] with coefficient 0.368020105
Adding x[C2, C1, V2] with coefficient 0.3675970924
Adding x[C2, C9, V2] with coefficient 0.3690071344
Adding x[C2, C8, V2] with coefficient 0.368255112
Adding x[C2, C12, V2] with coefficient 0.3688661302
Adding x[C2, C11, V2] with coefficient 0.3681611092
Adding x[C2, C13, V2] with coefficient 0.3678791008
Adding x[C2, C3, V2] with coefficient 0.3665160602
Adding x[C2, C5, V2] with coefficient 0.3691011372
Adding x[C2, C7, V2] with coefficient 0.3667510672
Adding x[C2, C6, V2] with coefficient 0.3682081106
Adding x[C2, C14, V2] with coefficient 0.3681141078
Adding x[C10, C2, V2] with coefficient 0.3661870504
Adding x[C10, C4, V2] with coefficient 0.3679731036
Adding x[C10, C15, V2] with coefficient 0.3683491148
Adding x[C10, C16, V2] with coefficient 0.3690071344
Adding x[C10, C1, V2

range(564, 565)

In [17]:
### Solve the model
model.parameters.mip.tolerances.mipgap.set(0.01)  # Set MIP gap tolerance
model.solve()

model.write("output.lp")
# Output the solution
solution = model.solution
if solution.is_primal_feasible():
    total_cost_all_vehicles = 0  # Initialize total cost for all vehicles
    total_emissions_all_vehicles = 0  # Initialize total emissions for all vehicles
    for k in K:
        route = []
        total_cost = 0  # Initialize total cost for the vehicle
        total_emissions = 0  # Initialize total emissions for the vehicle
        for i in locations:
            print(f"u[{i}, {k}] = {solution.get_values(u[(i, k)])}")
            print(f"--------")
            for j in locations:
                if i != j and solution.get_values(x[(i, j, k)]) > 0.5:
                    route.append((i, j))
                    cost = cij[(i, j, k)]
                    emissions = eij[(i, j, k)]
                    total_cost += cost
                    total_emissions += emissions
                    print(f"Route taken: c[{i}, {j}, {k}] = {cost}, e[{i}, {j}, {k}] = {emissions}")
                    print(f"--------")
        if route:
            print(f"Vehicle {k} route: {route}")
            print("--------")
            print(f"Total cost for vehicle {k}: {total_cost}")
            print(f"Total emissions for vehicle {k}: {total_emissions}")
        total_cost_all_vehicles += total_cost
        total_emissions_all_vehicles += total_emissions
    print(f"--------")
    print(f"Total cost for all vehicles: {total_cost_all_vehicles}")
    print(f"Total emissions for all vehicles: {total_emissions_all_vehicles}")
else:
    print("No feasible solution found.")

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
Tried aggregator 1 time.
MIP Presolve eliminated 32 rows and 36 columns.
MIP Presolve modified 60 coefficients.
Reduced MIP has 533 rows, 576 columns, and 3552 nonzeros.
Reduced MIP has 544 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.73 ticks)
Probing time = 0.00 sec. (2.56 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 533 rows, 576 columns, and 3552 nonzeros.
Reduced MIP has 544 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.09 ticks)
Probing time = 0.00 sec. (2.54 ticks)
Clique table members: 2205.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.02 sec. (1.23 ticks)

        Nodes                                      

u[C2, V2] = 89.70945600000002
--------
Route taken: c[C2, C10, V2] = 27.0, e[C2, C10, V2] = 0.3661870504
--------
u[C11, V2] = 15.019228999999996
--------
Route taken: c[C11, C13, V2] = 15.75, e[C11, C13, V2] = 0.3654820294
--------
u[C13, V2] = 15.010037000000011
--------
Route taken: c[C13, C1, V2] = 14.25, e[C13, C1, V2] = 0.3653880266
--------
u[C5, V2] = 41.890949000000006
--------
Route taken: c[C5, C6, V2] = 29.25, e[C5, C6, V2] = 0.3663280546
--------
u[C6, V2] = 53.743908000000005
--------
Route taken: c[C6, C3, V2] = 21.0, e[C6, C3, V2] = 0.3658110392
--------
u[C15, V2] = 0.0
--------
u[C4, V2] = 0.0
--------
u[C16, V2] = 0.0
--------
u[W1, V2] = 0.0
--------
Route taken: c[W1, C14, V2] = 64.5, e[W1, C14, V2] = 0.3685371204
--------
u[C1, V2] = 30.400014999999996
--------
Route taken: c[C1, C5, V2] = 29.25, e[C1, C5, V2] = 0.3663280546
--------
u[C9, V2] = 2.0571319999999957
--------
Route taken: c[C9, C11, V2] = 9.75, e[C9, C11, V2] = 0.3651060182
--------
u[C8, V2] = 0.0
-